# End-to-End Example for BigQuery TensorFlow Reader

## Overview

In this lab, you will use [BigQuery TensorFlow reader](https://github.com/tensorflow/io/tree/master/tensorflow_io/bigquery) and Keras sequential API to train a neural network.

## Learning Objective

* Import census data into BigQuery.
* Load census data in TensorFlow dataset using BigQuery reader.
* Build and train a model.
* Evaluate the model.

## Introduction

This tutorial shows how to use [BigQuery TensorFlow reader](https://github.com/tensorflow/io/tree/master/tensorflow_io/bigquery) and Keras sequential API to train a neural network.

Each learning objective will correspond to a __#TODO__ in this student lab notebook -- try to complete this notebook first and then review the [solution notebook](../solutions/bigquery.ipynb).

**Make sure to enable the BigQueryStorage API.**

## Setup

Install required Packages, and restart runtime

In [1]:
try:
  # Use the Colab's preinstalled TensorFlow 2.x
  %tensorflow_version 2.x 
except:
  pass

In [2]:
# Install the required packages
!pip install fastavro
!pip install tensorflow-io==0.9.0

     |████████████████████████████████| 41.6 MB 6.7 MB/s eta 0:00:01
     |████████████████████████████████| 86.4 MB 63.9 MB/s eta 0:00:01
     |████████████████████████████████| 50 kB 8.1 MB/s  eta 0:00:01
     |████████████████████████████████| 449 kB 51.1 MB/s eta 0:00:01
     |████████████████████████████████| 3.8 MB 54.4 MB/s eta 0:00:01
     |████████████████████████████████| 2.9 MB 47.7 MB/s eta 0:00:01
     |████████████████████████████████| 20.1 MB 43.5 MB/s eta 0:00:01    |████████████▏                   | 7.7 MB 43.5 MB/s eta 0:00:01
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=c9a717d55a7fdcdf6253c869e440e30a027cba92df7ebc2d6689e5a888e9afd4
  Stored in directory: /home/jupyter/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
  Attempting u

**Please ignore the incompatible errors.**

In [3]:
# Install the specified package
!pip install google-cloud-bigquery-storage

Set your PROJECT ID

In [4]:
PROJECT_ID = "<YOUR PROJECT>" #@param {type:"string"}
! gcloud config set project $PROJECT_ID
%env GCLOUD_PROJECT=$PROJECT_ID

Updated property [core/project].
env: GCLOUD_PROJECT=qwiklabs-gcp-04-69798a999f35


Import Python libraries, define constants

In [5]:
# Import necessary libraries
from __future__ import absolute_import, division, print_function, unicode_literals

import os
from six.moves import urllib
import tempfile

import numpy as np
import pandas as pd
import tensorflow as tf

from google.cloud import bigquery
from google.api_core.exceptions import GoogleAPIError

LOCATION = 'us'

# Storage directory
DATA_DIR = os.path.join(tempfile.gettempdir(), 'census_data')

# Download options.
DATA_URL = 'https://storage.googleapis.com/cloud-samples-data/ml-engine/census/data'
TRAINING_FILE = 'adult.data.csv'
EVAL_FILE = 'adult.test.csv'
TRAINING_URL = '%s/%s' % (DATA_URL, TRAINING_FILE)
EVAL_URL = '%s/%s' % (DATA_URL, EVAL_FILE)

DATASET_ID = 'census_dataset'
TRAINING_TABLE_ID = 'census_training_table'
EVAL_TABLE_ID = 'census_eval_table'

CSV_SCHEMA = [
      bigquery.SchemaField("age", "FLOAT64"),
      bigquery.SchemaField("workclass", "STRING"),
      bigquery.SchemaField("fnlwgt", "FLOAT64"),
      bigquery.SchemaField("education", "STRING"),
      bigquery.SchemaField("education_num", "FLOAT64"),
      bigquery.SchemaField("marital_status", "STRING"),
      bigquery.SchemaField("occupation", "STRING"),
      bigquery.SchemaField("relationship", "STRING"),
      bigquery.SchemaField("race", "STRING"),
      bigquery.SchemaField("gender", "STRING"),
      bigquery.SchemaField("capital_gain", "FLOAT64"),
      bigquery.SchemaField("capital_loss", "FLOAT64"),
      bigquery.SchemaField("hours_per_week", "FLOAT64"),
      bigquery.SchemaField("native_country", "STRING"),
      bigquery.SchemaField("income_bracket", "STRING"),
  ]

UNUSED_COLUMNS = ["fnlwgt", "education_num"]

## Import census data into BigQuery

Define helper methods to load data into BigQuery

In [6]:
def create_bigquery_dataset_if_necessary(dataset_id):
  # Construct a full Dataset object to send to the API.
  client = bigquery.Client(project=PROJECT_ID)
  dataset = bigquery.Dataset(bigquery.dataset.DatasetReference(PROJECT_ID, dataset_id))
  dataset.location = LOCATION

  try:
    # Constructs the API request
    dataset = # TODO -- Your code goes here
    return True
  except GoogleAPIError as err:
    if err.code != 409: # http_client.CONFLICT
      raise
  return False


In [7]:
def load_data_into_bigquery(url, table_id):
  create_bigquery_dataset_if_necessary(DATASET_ID)
  client = bigquery.Client(project=PROJECT_ID)
  dataset_ref = client.dataset(DATASET_ID)
  table_ref = dataset_ref.table(table_id)
  job_config = bigquery.LoadJobConfig()
  job_config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE
  job_config.source_format = bigquery.SourceFormat.CSV
  job_config.schema = CSV_SCHEMA

  # Constructs the Job to load data into table
  load_job = # TODO -- Your code goes here(
      url, table_ref, job_config=job_config
  )
  print("Starting job {}".format(load_job.job_id))

  load_job.result()  # Waits for table load to complete.
  print("Job finished.")

  destination_table = client.get_table(table_ref)
  print("Loaded {} rows.".format(destination_table.num_rows))

Load Census data in BigQuery.

In [8]:
load_data_into_bigquery(TRAINING_URL, TRAINING_TABLE_ID)
load_data_into_bigquery(EVAL_URL, EVAL_TABLE_ID)

Starting job 74b20d98-d84f-4560-b59c-ac8e4b14a0d3
Job finished.
Loaded 32561 rows.
Starting job f1cc2def-057d-4320-9d19-e0885a024566
Job finished.
Loaded 16278 rows.


Confirm that data was imported

TODO: replace \<YOUR PROJECT\> with your PROJECT_ID

Note: --use_bqstorage_api will get data using BigQueryStorage API and will make sure that you are authorized to use it. Make sure that it is enabled for your project: https://cloud.google.com/bigquery/docs/reference/storage/#enabling_the_api


In [9]:
%%bigquery --use_bqstorage_api
SELECT * FROM `<YOUR PROJECT>.census_dataset.census_training_table` LIMIT 5

Downloading: 100%|██████████| 5/5 [00:01<00:00,  3.31rows/s]


,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income_bracket
0,39.0,Private,297847.0,9th,5.0,Married-civ-spouse,Other-service,Wife,Black,Female,3411.0,0.0,34.0,United-States,<=50K
1,72.0,Private,74141.0,9th,5.0,Married-civ-spouse,Exec-managerial,Wife,Asian-Pac-Islander,Female,0.0,0.0,48.0,United-States,>50K
2,45.0,Private,178215.0,9th,5.0,Married-civ-spouse,Machine-op-inspct,Wife,White,Female,0.0,0.0,40.0,United-States,>50K
3,31.0,Private,86958.0,9th,5.0,Married-civ-spouse,Exec-managerial,Wife,White,Female,0.0,0.0,40.0,United-States,<=50K
4,55.0,Private,176012.0,9th,5.0,Married-civ-spouse,Tech-support,Wife,White,Female,0.0,0.0,23.0,United-States,<=50K


## Load census data in TensorFlow DataSet using BigQuery reader

Read and transform cesnus data from BigQuery into TensorFlow DataSet

In [10]:
from tensorflow.python.framework import ops
from tensorflow.python.framework import dtypes
from tensorflow_io.bigquery import BigQueryClient
from tensorflow_io.bigquery import BigQueryReadSession
  
def transform_row(row_dict):
  # Trim all string tensors
  trimmed_dict = { column:
                  (tf.strings.strip(tensor) if tensor.dtype == 'string' else tensor) 
                  for (column,tensor) in row_dict.items()
                  }
  # Extract feature column
  income_bracket = trimmed_dict.pop('income_bracket')
  # Convert feature column to 0.0/1.0
  income_bracket_float = tf.cond(tf.equal(tf.strings.strip(income_bracket), '>50K'), 
                 lambda: tf.constant(1.0), 
                 lambda: tf.constant(0.0))
  return (trimmed_dict, income_bracket_float)

def read_bigquery(table_name):
  tensorflow_io_bigquery_client = BigQueryClient()
  read_session = tensorflow_io_bigquery_client.read_session(
      "projects/" + PROJECT_ID,
      PROJECT_ID, table_name, DATASET_ID,
      list(field.name for field in CSV_SCHEMA 
           if not field.name in UNUSED_COLUMNS),
      list(dtypes.double if field.field_type == 'FLOAT64' 
           else dtypes.string for field in CSV_SCHEMA
           if not field.name in UNUSED_COLUMNS),
      requested_streams=2)
  
  # Read the rows in parallel streams
  dataset = # TODO -- Your code goes here
  # Apply transformation to the dataset
  transformed_ds = # TODO -- Your code goes here(transform_row)
  return transformed_ds


In [11]:
BATCH_SIZE = 32

training_ds = read_bigquery(TRAINING_TABLE_ID).shuffle(10000).batch(BATCH_SIZE)
eval_ds = read_bigquery(EVAL_TABLE_ID).batch(BATCH_SIZE)

2021-09-10 08:16:55.680866: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2021-09-10 08:16:55.686447: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2200155000 Hz
2021-09-10 08:16:55.687680: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x5635177c2920 executing computations on platform Host. Devices:
2021-09-10 08:16:55.687727: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): Host, Default Version


Instructions for updating:
Use `tf.data.Dataset.interleave(map_func, cycle_length, block_length, num_parallel_calls=tf.data.experimental.AUTOTUNE)` instead. If sloppy execution is desired, use `tf.data.Options.experimental_determinstic`.


## Define feature columns

In [12]:
def get_categorical_feature_values(column):
  query = 'SELECT DISTINCT TRIM({}) FROM `{}`.{}.{}'.format(column, PROJECT_ID, DATASET_ID, TRAINING_TABLE_ID)
  client = bigquery.Client(project=PROJECT_ID)
  dataset_ref = client.dataset(DATASET_ID)
  job_config = bigquery.QueryJobConfig()
  query_job = client.query(query, job_config=job_config)
  result = query_job.to_dataframe()
  return result.values[:,0]

In [13]:
from tensorflow import feature_column

feature_columns = []

# numeric cols
for header in ['capital_gain', 'capital_loss', 'hours_per_week']:
  feature_columns.append(feature_column.numeric_column(header))

# categorical cols
for header in ['workclass', 'marital_status', 'occupation', 'relationship',
               'race', 'native_country', 'education']:
  categorical_feature = feature_column.categorical_column_with_vocabulary_list(
        header, get_categorical_feature_values(header))
  categorical_feature_one_hot = feature_column.indicator_column(categorical_feature)
  feature_columns.append(categorical_feature_one_hot)

# bucketized cols
age = feature_column.numeric_column('age')
age_buckets = feature_column.bucketized_column(age, boundaries=[18, 25, 30, 35, 40, 45, 50, 55, 60, 65])
feature_columns.append(age_buckets)

feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

## Build and train model

Build model

In [14]:
Dense = tf.keras.layers.Dense
model = tf.keras.Sequential(
  [
    feature_layer,
      Dense(100, activation=tf.nn.relu, kernel_initializer='uniform'),
      Dense(75, activation=tf.nn.relu),
      Dense(50, activation=tf.nn.relu),
      Dense(25, activation=tf.nn.relu),
      Dense(1, activation=tf.nn.sigmoid)
  ])

# Compile Keras model
model.compile(
    loss='binary_crossentropy', 
    metrics=['accuracy'])

Train model

In [15]:
# Train the model
# TODO -- Your code goes here(training_ds, epochs=5)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Epoch 1/5
1018/1018 [==============================] - 13s 13ms/step - loss: 0.5052 - accuracy: 0.8038
Epoch 2/5


2021-09-10 08:18:01.591898: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]


1018/1018 [==============================] - 10s 9ms/step - loss: 0.3776 - accuracy: 0.8166
Epoch 3/5


2021-09-10 08:18:11.181031: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]


1018/1018 [==============================] - 9s 9ms/step - loss: 0.3613 - accuracy: 0.8306
Epoch 4/5


2021-09-10 08:18:20.400916: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]


1018/1018 [==============================] - 9s 9ms/step - loss: 0.3469 - accuracy: 0.8391
Epoch 5/5


2021-09-10 08:18:29.751204: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]


1018/1018 [==============================] - 9s 9ms/step - loss: 0.3414 - accuracy: 0.8409


2021-09-10 08:18:39.025880: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]


## Evaluate model

Evaluate model

In [16]:
# Evaluate the model
loss, accuracy = # TODO -- Your code goes here(eval_ds)
print("Accuracy", accuracy)

509/509 [==============================] - 6s 11ms/step - loss: 0.3419 - accuracy: 0.8313
Accuracy 0.83130604


2021-09-10 08:18:59.583508: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]


Evaluate a couple of random samples

In [17]:
sample_x = {
    'age' : np.array([56, 36]), 
    'workclass': np.array(['Local-gov', 'Private']), 
    'education': np.array(['Bachelors', 'Bachelors']), 
    'marital_status': np.array(['Married-civ-spouse', 'Married-civ-spouse']), 
    'occupation': np.array(['Tech-support', 'Other-service']), 
    'relationship': np.array(['Husband', 'Husband']), 
    'race': np.array(['White', 'Black']), 
    'gender': np.array(['Male', 'Male']), 
    'capital_gain': np.array([0, 7298]), 
    'capital_loss': np.array([0, 0]), 
    'hours_per_week': np.array([40, 36]), 
    'native_country': np.array(['United-States', 'United-States'])
  }

model.predict(sample_x)

array([[0.6854977],
       [0.940263 ]], dtype=float32)